In [ ]:
%%writefile decor_header.py
import random; from IPython.display import display,HTML
from IPython.core.magic import register_line_magic
@register_line_magic
def decor_header(params):
    params=params.split('|'); string=params[0]
    if len(params)==1: 
        font_size='22'; font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==2: 
        font_size=params[1]
        font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==3: 
        font_size=params[1]; font_family=params[2]
        cmap='Sinebow'
    else: 
        font_size=params[1]; font_family=params[2]; cmap=params[3]
    height=max([int(font_size)*2.5,60]) 
    randi=str(random.uniform(0,9999999))
    html_str="""
<script src='https://d3js.org/d3.v6.min.js'></script>
<style>
@import 'https://fonts.googleapis.com/css?family="""+font_family+"""';
#colorized001 {
font-family:"""+font_family+"""; font-size:"""+font_size+""";}
#canvas001,#canvas002 {width:10%; vertical-align:middle;}
</style>
<text id='colorized001'><canvas id='canvas001'></canvas>
"""+string+"""
<canvas id='canvas002'></canvas></text><br/>
<script>
var tc=setInterval(function() {
    var now=(new Date().getTime()%5000)/5000;
    var now_slow=(new Date().getTime()%100000)/100000;
    var iddoc=document.getElementById('colorized001');
    iddoc.style.color=d3.interpolate"""+cmap+"""(now);
    var r=10,n=7;
    var c1=document.getElementById('canvas001'); 
    var context1=c1.getContext('2d');
    var c2=document.getElementById('canvas002'); 
    var context2=c2.getContext('2d');
    c1.style.background=d3.interpolate"""+cmap+"""(now_slow); 
    c2.style.background=d3.interpolate"""+cmap+"""(now_slow);
    context1.strokeStyle=d3.interpolate"""+cmap+"""(now);    
    context2.strokeStyle=d3.interpolate"""+cmap+"""(now);
    for (var i=1; i<n; i++) {
        context1.beginPath(); context2.beginPath();
        for (var j=0; j<6; j++) {
            context1.arc(60*j,r*(n+.5),i*r,0,2*Math.PI);
            context2.arc(60*j,r*(n+.5),i*r,0,2*Math.PI); };
        context1.stroke(); context2.stroke(); }; },1)
</script>"""
    file='d3header'+randi+'.html'
    with open(file,'w') as f:
         f.write(html_str); f.close()
    string="""<div id='html_string"""+randi+\
    """' style='width:100%;'><iframe src='"""+file+\
    """' height="""+str(height)+"""
    style='display:block; width:100%;'></iframe></div>"""
    display(HTML(string))

In [ ]:
%run decor_header.py
%decor_header Modules, Helpful Functions, & Styling

In [ ]:
!pip install --upgrade neural_structured_learning --user --quiet 

In [ ]:
%%html
<style>
@import 'https://fonts.googleapis.com/css?family=Roboto';
span {text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Roboto; font-size:110%; color:#36ff36;}      
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import h5py,pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split
file_path3='../input/classification-of-handwritten-letters/'
file_path4='../input/flower-color-images/'
model_weights='/checkpoints'

In [ ]:
def prepro(x_train,y_train,x_test,y_test):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape]],
                    columns=['train','valid','test'],
                    index=['image arrays','label arrays'])
    display(df.style.set_table_styles(pd_style()))
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test]]
def pd_style():
    return [dict(selector='th',
                 props=[('font-size','12pt'),('min-width','150px')]),
            dict(selector='td',
                 props=[('padding','0em 0em'),('min-width','150px')]),
            dict(selector='tr:hover th:hover',
                 props=[('font-size','14pt'),('max-width','150px'),
                        ('text-shadow','3px 3px 3px #aaa')]),
            dict(selector='tr:hover td:hover',
                 props=[('font-size','12pt'),('max-width','150px'),
                        ('text-shadow','3px 3px 3px #aaa')])]
def cb(mw):
    early_stopping=tf.keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        patience=20,verbose=2,mode='max')
    checkpointer=tf.keras.callbacks.ModelCheckpoint(
        save_best_only=True,save_weights_only=True,
        monitor='val_sparse_categorical_accuracy',
        filepath=mw,verbose=2,mode='max')
    lr_reduction=tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_sparse_categorical_accuracy',
        verbose=2,patience=5,factor=.8,mode='max')
    return [checkpointer,early_stopping,lr_reduction]

In [ ]:
%decor_header Data Loading & Preprocessing

In [ ]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1))

In [ ]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2)

In [ ]:
f=h5py.File(file_path3+'LetterColorImages_123.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[1]]).astype('float32')/255
labels=np.array(f[keys[2]]).astype('int32').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3)

In [ ]:
f=h5py.File(file_path4+'FlowerColorImages.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[0]]).astype('float32')/255
labels=np.array(f[keys[1]]).astype('int32').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4)

In [ ]:
%decor_header Models with Adversarial Regularization
%decor_header MLP Base|18

In [ ]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs.make_adv_reg_config(
    multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras.AdversarialRegularization(
    base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test1,'label':y_test1},verbose=0)))

In [ ]:
batch_size=64; img_size=32; n_class=10; epochs=7
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test2,'label':y_test2},verbose=0)))

In [ ]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test3,'label':y_test3},verbose=0)))

In [ ]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test4,'label':y_test4},verbose=0)))

In [ ]:
%decor_header CNN Base|18

In [ ]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test1.reshape(x_test1.shape[0],28,28,1),
              'label':y_test1},verbose=0)))

In [ ]:
batch_size=64; img_size=32; n_class=10; epochs=20
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test2,'label':y_test2},verbose=0)))

In [ ]:
batch_size=128; img_size=32; n_class=33; epochs=150
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test3,'label':y_test3},verbose=0)))

In [ ]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [ ]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

In [ ]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test4,'label':y_test4},verbose=0)))